In [ ]:
!pip install sentence_transformers
!pip install pinecone-client
!pip install protobuf
!pip install transformers
!pip install pandas
!pip install sentencepiece 

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from pinecone import Pinecone, ServerlessSpec

/opt/mamba/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
model = SentenceTransformer("dangvantuan/sentence-camembert-large")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with mean pooling.


In [3]:
pc = Pinecone(api_key="")

# service public

In [ ]:
sentences = df['key_content'].values
for i, line in enumerate(sentences):
  if i % 100 == 0:
    print(i)
  sentences[i] = model.encode(line)

In [ ]:
vectors = []

for i, row in df.iterrows():
    vectors.append({
        "id": str(i),
        "values": sentences[i],
        "metadata": {
            "text": row["key_content"],
            "url": row['url'],
            "title": row["title"]
        }
    })


In [ ]:
import more_itertools

# Function to split the vectors list into chunks of a specified size
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Create the index
index = pc.Index("gouvx")

# Split the vectors into chunks of 100
vector_chunks = chunks(vectors, 100)

# Upsert each chunk
for chunk in vector_chunks:
    index.upsert(
        vectors=chunk,
        namespace="servicepublic"
    )

In [ ]:
# update title field
index = pc.Index("gouvx")

titles = df['title'].values
for i, title in enumerate(titles):
  if i % 100 == 0:
    print(i)
  index.update(namespace="servicepublic", id=str(i), set_metadata={"title": title})

# Legifrance

In [6]:
!wget https://storage.googleapis.com/gouvx/legifrance/legifrance.csv

--2024-07-03 09:01:03--  https://storage.googleapis.com/gouvx/legifrance/legifrance.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.178.155, 142.250.179.123, 142.250.75.251, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.178.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233803020 (223M) [text/csv]
Saving to: ‘legifrance.csv’

legifrance.csv      100%[===================>] 222.97M  68.4MB/s    in 3.4s    

2024-07-03 09:01:07 (65.3 MB/s) - ‘legifrance.csv’ saved [233803020/233803020]



In [4]:
import pandas as pd
df = pd.read_csv("legifrance.csv")

In [5]:
def recreate_title(proper_title):
  title = proper_title.split(' > ')
  title.reverse()
  return " - ".join(title) + " - Légifrance"

def recreate_text(row):
  return f"""{row['hierarchie']}
{row['article_content']}"""

In [6]:
df_no_annex = df[~df["article_ref"].str.contains("Annexe")].copy()
df_no_annex['title'] = df_no_annex['proper_title'].apply(lambda x: recreate_title(x))
df_no_annex['text'] = df_no_annex.apply(lambda x: recreate_text(x), axis=1)
df = df_no_annex[['title', 'text', 'url', 'domain', 'subdomain']]

In [7]:
df["token_len"] = df["text"].apply(lambda x: len(model.tokenizer.encode(x)))

In [ ]:
df_large = df[df["token_len"] > 512]
df_small = df[df["token_len"] <= 512]

print(len(df), len(df_large), len(df_small))
print(len(df_large)/len(df), len(df_small)/len(df))

150650 11605 139045
0.07703285761699304 0.9229671423830069


In [ ]:
import tqdm

sentences = df_small['text'].values.copy()
#for i, line in enumerate(sentences):
for i, line in tqdm.tqdm(enumerate(sentences)):
  sentences[i] = model.encode(line)

52507it [54:32, 13.49it/s]

In [ ]:
vectors = []

for i, row in df.iterrows():
    vectors.append({
        "id": str(i),
        "values": sentences[i],
        "metadata": {
            "text": row["text"],
            "url": row['url'],
            "title": row["title"]
        }
    })


NameError: name 'df' is not defined

In [ ]:
import more_itertools

# Function to split the vectors list into chunks of a specified size
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Create the index
index = pc.Index("gouvx")

# Split the vectors into chunks of 100
vector_chunks = chunks(vectors, 100)

# Upsert each chunk
for chunk in vector_chunks:
    index.upsert(
        vectors=chunk,
        namespace="legifrance"
    )